# Imports

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import json
import random
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats

# Extract cell activity and trial variables from simulation results

In [2]:
# Initialize arrays of interest variables 
# NOTE: each neuron's activity is a row; column index is trial number

N_PFC_NEURONS = 500
N_MD_NEURONS = 2
N_OUTPUT_NEURONS = 2
TRIAL_TIME = 200

s_trial = 0
t_trial = 4800
n_trials = t_trial - s_trial

rPFC_trials = np.zeros((n_trials, TRIAL_TIME, N_PFC_NEURONS))
task_input = np.zeros((2, n_trials))

# Extract the data

data_dir = "/om2/group/halassa/PFCMD-ali-sabrina/020321_qvals/by_trial"

for i in range(s_trial, t_trial):
    with open(data_dir + "/" + str(i) + ".json") as json_file:
        data = json.load(json_file)
        idx = i - s_trial
                
        rPFC = np.array(data["network_rates"]["r_PFC"])
        rPFC_trials[idx,:,:] = rPFC
        
        trial_task_input = np.array(data["trial_data"]["input"])
        task_input[:,idx] = trial_task_input[:2]


# Compute logistic regression seudo p-squared values for all PFC neurons

## 1. Get an equal number of trials per association levels 
Note the association levels are:
`[.90, .10, .90, .70, .90, .10, .70, .30, .90, .50, .90, .10]`

We will get 50 trials from .90, .70, .30 and .10, and we will get 100 trials from .50.
We only consider trials in the second half of the association level, when learning is finished and activity is more likely to be stable.

In [58]:
# N_PFC_NEURONS = 500
CUE_TIME = 100

alevel_90_idxs = [0, 2, 4, 8, 10]
alevel_10_idxs = [1, 5, 11]
alevel_70_idxs = [3, 6]
alevel_30_idxs = [7]
alevel_50_idxs = [9]

def sample_trials(alevel_idxs, n_samples):
    trial_per_alevel = 400
    trials_offset = 200 # Only consider the second half of the alevel
    
    trials_to_sample = []
    for n_bin in alevel_idxs:
        trial_s = n_bin * trial_per_alevel + trials_offset
        trial_t = (n_bin + 1) * 400
        trials_to_sample = trials_to_sample + list(range(trial_s, trial_t))
    return random.sample(trials_to_sample, n_samples)
    
prsquareds = []    
for neuron_idx in range(N_PFC_NEURONS):
    
    # Randomly sample trials for analysis from the second half of each alevel
    # We ensure all alevels are equally represented
    
    alevel_90_trials = sample_trials(alevel_90_idxs, 100)
    alevel_10_trials = sample_trials(alevel_10_idxs, 100)
    alevel_70_trials = sample_trials(alevel_70_idxs, 100)
    alevel_30_trials = sample_trials(alevel_30_idxs, 100)
    alevel_50_trials = sample_trials(alevel_50_idxs, 200)
    trials = alevel_90_trials + alevel_10_trials + alevel_70_trials + alevel_30_trials + alevel_50_trials

    # Get cue 1 value and neuron activity for each trial
    # Only consider the activity over the second half of the cue period
    
    cuetime_s = math.floor(CUE_TIME - (CUE_TIME / 2))
    cuetime_t = CUE_TIME
    
    cue = task_input[0, trials]
    neuron_activity = np.mean(rPFC_trials[trials, cuetime_s:cuetime_t, neuron_idx],1)
    
    # Compute linear regression
    
    # Check if data is prefectly seperable 
    range_cue0 = (min(neuron_activity[cue == 0]), max(neuron_activity[cue == 0])) 
    is_separable = not any([x >= range_cue0[0] and x <= range_cue0[1] for x in neuron_activity[cue == 1]])
    if is_separable:
        prsquareds.append(1)
        continue
    
    # If not linearly separable run logistic regression
    
    X = np.transpose([np.ones(len(neuron_activity)), neuron_activity])
    y = cue
    model_fit = sm.Logit(y,X).fit(maxiter=50)
    prsquareds.append(model_fit.prsquared)


Optimization terminated successfully.
         Current function value: 0.625958
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.007779
         Iterations 17
Optimization terminated successfully.
         Current function value: 0.473914
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.022877
         Iterations 15
Optimization terminated successfully.
         Current function value: 0.283611
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.370539
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.453074
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.543881
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.564289
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.281331

Optimization terminated successfully.
         Current function value: 0.586503
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.631289
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.213389
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.014914
         Iterations 15
Optimization terminated successfully.
         Current function value: 0.546916
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.545827
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.499920
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.681960
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.549143
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.616198
 

/cm/shared/openmind/anaconda/3-2019.10/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Optimization terminated successfully.
         Current function value: 0.640101
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.681120
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.018000
         Iterations 14
Optimization terminated successfully.
         Current function value: 0.680953
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.294195
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.564905
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.690693
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.560184
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.656676
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.602198
 

Optimization terminated successfully.
         Current function value: 0.354810
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.550433
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.095253
         Iterations 11
Optimization terminated successfully.
         Current function value: 0.609264
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.690682
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.677794
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.262709
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.600227
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.667191
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691216
 

Optimization terminated successfully.
         Current function value: 0.374409
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.322826
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.399131
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.089352
         Iterations 11
Optimization terminated successfully.
         Current function value: 0.015638
         Iterations 14
Optimization terminated successfully.
         Current function value: 0.395322
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.650989
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.625245
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.341416
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.423157


/cm/shared/openmind/anaconda/3-2019.10/lib/python3.7/site-packages/statsmodels/discrete/discrete_model.py:1736: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/cm/shared/openmind/anaconda/3-2019.10/lib/python3.7/site-packages/statsmodels/discrete/discrete_model.py:1789: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


LinAlgError: Singular matrix

# Plot results
## 1. Plot histogram of r-squared values
## 2. Plot individual neuron activity with r-squared >= 0.95

In [60]:
SAVE_DIR = '/om2/group/halassa/PFCMD-ali-sabrina/030821_input-like'

# Plot distribution of pseudo r-squared values
plt.hist(prsquareds, bins=50)
plt.title("PFC neurons logistic regression r-squared values")
plt.xlabel("R^2 (cue vs neural activity)")
plt.ylabel("Number of neurons")
plt.savefig(f"{SAVE_DIR}/regression-histogram.jpg", transparent=False)
plt.close()

# Plot activity of neurons with r-squared >= 0.95

def plot_stderr(ax, x, c, l):
    m = np.mean(x, 0)
    stderr = stats.sem(x, 0)
    ax.plot(m, c, label=l)
    ax.plot(m + stderr, 'grey')
    ax.plot(m - stderr, 'grey')

neurons_of_interest = np.where(np.array(prsquareds) > 0.95)[0]
for neuron_idx in neurons_of_interest:    
    # Sample trials 
    alevel_90_trials = sample_trials(alevel_90_idxs, 100)
    alevel_10_trials = sample_trials(alevel_10_idxs, 100)
    alevel_70_trials = sample_trials(alevel_70_idxs, 100)
    alevel_30_trials = sample_trials(alevel_30_idxs, 100)
    alevel_50_trials = sample_trials(alevel_50_idxs, 200)
    trials = alevel_90_trials + alevel_10_trials + alevel_70_trials + alevel_30_trials + alevel_50_trials

    # Get cue information and neuron activity for trials
    cue = task_input[0, trials]
    neuron_activity = rPFC_trials[trials, :, neuron_idx]
    
    # Run the model fitting
    cuetime_s = math.floor(CUE_TIME - (CUE_TIME / 2))
    cuetime_t = CUE_TIME
    
    cue = task_input[0, trials]
    m_neuron_activity = np.mean(rPFC_trials[trials, cuetime_s:cuetime_t, neuron_idx],1)
    
    # Compute linear regression
    
    # Check if data is prefectly seperable 
    range_cue0 = (min(m_neuron_activity[cue == 0]), max(m_neuron_activity[cue == 0])) 
    is_separable = not any([x > range_cue0[0] and x < range_cue0[1] for x in m_neuron_activity[cue == 1]])
    if is_separable:
        prsquared = 1
    else:
        # If not linearly separable run logistic regression
        X = np.transpose([np.ones(len(m_neuron_activity)), m_neuron_activity])
        y = cue
        model_fit = sm.Logit(y,X).fit(maxiter=50)
        prsquared = model_fit.prsquared
        
    fig = plt.figure(figsize=(12,5))
    
    ax1 = fig.add_subplot(1, 2, 1)
    plot_stderr(ax1, neuron_activity[cue == 0], 'b', 'cue=0')
    plot_stderr(ax1, neuron_activity[cue == 1], 'r', 'cue=1')
    ax1.axvline(CUE_TIME, color='grey', linestyle=':')
    ax1.legend()
    ax1.set_title(f"Neural activity")
    ax1.set_xlabel("Trial time")
    ax1.set_ylabel("Neuron activity (m +/- sem)")
    
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.scatter(m_neuron_activity, cue)
    if prsquared < 1:
        points = sorted(zip(m_neuron_activity, model_fit.predict(X)) , key=lambda k: [k[1], k[0]])
        points_x, points_y = map(list, zip(*points))
        ax2.plot(points_x, points_y, 'k')
    ax2.set_xlabel("Mean activity in cue (second half)")
    ax2.set_ylabel("Cue")
    ax2.set_title("Logistic regression")
    fig.suptitle(f"Neuron {neuron_idx}, r-squared={prsquareds[neuron_idx]}")
    
    plt.savefig(f"{SAVE_DIR}/nidx={neuron_idx}.jpg", transparent=False)
    plt.close()

63


SyntaxError: 'return' outside function (<ipython-input-60-a47d65684ade>, line 14)